In [1]:
import numpy
import typing
import sys
sys.path.append('..')

In [2]:
def convolve1D_circular(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = (i - (kernel.shape[0]//2 - 1) + k) % signal.shape[0]
            k_idx = kernel.shape[0]-1-k
            out[i] += signal[s_idx] * kernel[k_idx]
        
    return out

def convolve1D_zero_padding(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = i - (kernel.shape[0]//2 - 1) + k
            if (0 <= s_idx) and (s_idx < signal.shape[0]):
                k_idx = kernel.shape[0]-1-k
                out[i] += signal[s_idx] * kernel[k_idx]
        
    return out

def convolve1D_padding_value(signal: numpy.ndarray, kernel: numpy.ndarray, padding_value: float) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = i - (kernel.shape[0]//2 - 1) + k
            if (0 <= s_idx) and (s_idx < signal.shape[0]):
                k_idx = kernel.shape[0]-1-k
                out[i] += signal[s_idx] * kernel[k_idx]
            else:
                k_idx = kernel.shape[0]-1-k
                out[i] += padding_value * kernel[k_idx]
        
    return out



def convolve1D_zero_padding_v2(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:

    out_shape = signal.shape[0] + kernel.shape[0] - 1
    out = numpy.zeros(shape=out_shape)

    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            s_idx = i - (kernel.shape[0]//2 - 1) + k
            if (0 <= s_idx) and (s_idx < signal.shape[0]):
                k_idx = kernel.shape[0]-1-k
                out[i] += signal[s_idx] * kernel[k_idx]
        
    return out






def convolve1D_full(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:
    
    out_shape = signal.shape[0] + kernel.shape[0] - 1
    
    to_convolve = numpy.zeros(shape=signal.shape[0] + 2*(kernel.shape[0]-1))
    to_convolve[kernel.shape[0]-1:-(kernel.shape[0]-1)] = signal
    
    out = numpy.zeros(shape=out_shape)
    reversed_kernel = kernel[::-1]
    
    for i in range(0, out_shape):
        for k in range(0, kernel.shape[0]):
            out[i] += to_convolve[i+k]*reversed_kernel[k]
            
    return out

    
def convolve2D_full(signal: numpy.ndarray, kernel: numpy.ndarray) -> numpy.ndarray:
    
    out_shape = [
        signal.shape[0] + kernel.shape[0] - 1,
        signal.shape[1] + kernel.shape[1] - 1
    ]
    
    to_convolve = numpy.zeros(
        shape = [
            signal.shape[0] + 2*(kernel.shape[0]-1),
            signal.shape[1] + 2*(kernel.shape[1]-1)
        ]
    )
    
    to_convolve[
        kernel.shape[0]-1:-(kernel.shape[0]-1),
        kernel.shape[1]-1:-(kernel.shape[1]-1)
    ] = signal
    
    out = numpy.zeros(shape=out_shape)
    reversed_kernel = kernel[::-1, ::-1]
    
    for i in range(0, out_shape[0]):
        for j in range(0, out_shape[1]):
            for k1 in range(0, kernel.shape[0]):
                for k2 in range(0, kernel.shape[1]):
                    out[i, j] += to_convolve[i+k1, j+k2]*reversed_kernel[k1, k2]
            
    return out

def gaussian1D(value: float, mu: float, sigma: float) -> typing.Union[float, numpy.ndarray]:
    out = numpy.exp( - (value - mu)**2 / (2*sigma**2) )
    # out /= ( 2*numpy.pi) ** (1/2)
    out /= numpy.sqrt(2*numpy.pi)
    out /= sigma
    return out
    
# @classmethod
# def gaussian2D(cls, x: float, y:float, mu_x: float, mu_y: float, sigma_x: float, sigma_y: float) -> float :
#     out_x = numpy.exp( - (x - mu_x)**2 / sigma_x**2 )
#     out_x /= ( 2*numpy.pi) ** (1/2) * sigma_x
#     out_y = numpy.exp( - (y - mu_y)**2 / sigma_y**2 )
#     out_y /= ( 2*numpy.pi) ** (1/2) * sigma_y
#     return out_x * out_y


def gaussian2D(x: float, y: float, mu_x: float, mu_y: float, sigma_x: float, sigma_y: float) -> typing.Union[float, numpy.ndarray]:
    gaussian_x = gaussian1D(x, mu_x, sigma_x)
    gaussian_y = gaussian1D(y, mu_y, sigma_y)
    return gaussian_x*gaussian_y
    

def gaussianND(xi: numpy.ndarray, mu: numpy.ndarray, sigma: numpy.ndarray) -> typing.Union[float, numpy.ndarray]:
    dim = len(xi)
    gauss1D =  [ gaussian1D(xi[i], mu[i], sigma[i]) for i in range(0, dim) ]
    out = numpy.multiply(*gauss1D)
    return out

In [3]:
size = numpy.array([3, 3])
dim = 2
params = [ numpy.arange(-size[i]//2+1, size[i]//2+1) for i in range(0, size.shape[0]) ]
xi = numpy.meshgrid(*params)
mu, sigma = 0.0, 1.0
gaussian2D(*xi, mu, mu, sigma, sigma)

array([[0.05854983, 0.09653235, 0.05854983],
       [0.09653235, 0.15915494, 0.09653235],
       [0.05854983, 0.09653235, 0.05854983]])

In [4]:
size = numpy.array([3, 3, 3])
params = [ numpy.arange(-size[i]//2+1, size[i]//2+1) for i in range(0, size.shape[0]) ]
xi = numpy.meshgrid(*params)
mu = numpy.array([0.0, 0.0, 0.0])
sigma = numpy.array([1.0, 1.0, 1.0])
gaussianND(xi, mu, sigma)

array([[[0.05854983, 0.05854983, 0.05854983],
        [0.09653235, 0.09653235, 0.09653235],
        [0.05854983, 0.05854983, 0.05854983]],

       [[0.09653235, 0.09653235, 0.09653235],
        [0.15915494, 0.15915494, 0.15915494],
        [0.09653235, 0.09653235, 0.09653235]],

       [[0.05854983, 0.05854983, 0.05854983],
        [0.09653235, 0.09653235, 0.09653235],
        [0.05854983, 0.05854983, 0.05854983]]])

In [5]:
import abc
import enum

class BoundaryCondition(enum.Enum):
    PERIODIC = 0
    CIRCULAR = 1
    ZEROS = 2

class ConvolveType(enum.Enum):
    FULL = 0
    SAME = 0


# def convolve_same(x: numpy.ndarray, kernel: numpy.ndarray, )


class Filter(abc.ABC):
        
    @abc.abstractmethod
    def __call__(self, x: numpy.ndarray) -> numpy.ndarray:
        pass


class GaussianFilter1D(Filter):
    
    def __init__(self, size: int, mu: float, sigma: float) -> None:
        x = numpy.arange(-size//2 + 1, size//2 + 1)
        self.kernel = gaussian1D(x, mu, sigma)
        self.__mu = mu
        self.__sigma = sigma

    @property
    def mu(self) -> float:
        return self.__mu
    
    @mu.setter
    def mu(self, value) -> float:
        raise AssertionError('You cant set mu !')
    
    @property
    def sigma(self) -> float:
        return self.__sigma
    
    @sigma.setter
    def sigma(self, value) -> float:
        raise AssertionError('You cant set sigma !')
        
    def __call__(self, x) -> numpy.array:
        return convolve1D_full(x, self.kernel)
        


# class SeparableGaussianFilterND:
    
#     def __init__(self, size: numpy.ndarray, sigmas: numpy.ndarray) -> None:
#         dim = size.shape[0]
#         params = numpy.array([ numpy.arange(-size[i]//2+1, size[i]//2+1) for i in range(0, dim) ])
#         xi = numpy.meshgrid(*params)
#         self.filters = [ Functional.gaussian1D(x, sigma) for x, sigma in zip(xi, sigmas) ]
        
#     def __call__(self, signal: float) -> numpy.array:
#         out = numpy.copy(signal)
#         for filter in self.filters:
#             out = filter(out)
#         return out
     

class GaussianFilter2D(Filter):
    
    def __init__(self, size: numpy.ndarray, mu: numpy.ndarray, sigma: numpy.ndarray) -> None:
        dim = size.shape[0]
        params = [ numpy.arange(-size[i]//2 + 1, size[i]//2 + 1) for i in range(0, dim) ]
        xi = numpy.meshgrid(*params)
        self.kernel = gaussian2D(*xi, *mu, *sigma)

    def __call__(self, x: numpy.ndarray) -> numpy.ndarray:
        return convolve2D_full(x, self.kernel)

class GaussianFilterND(Filter):
    
    def __init__(self, size: numpy.ndarray, mu: numpy.ndarray, sigma: numpy.ndarray) -> None:
        dim = size.shape[0]
        params = [ numpy.arange(-size[i]//2 + 1, size[i]//2 + 1) for i in range(0, dim) ]
        xi = numpy.meshgrid(*params)
        self.kernel = gaussianND(xi, mu, sigma)


# class SequentialFilter:

#     def __init__(self, filters: list[filter]) -> None:
    

SyntaxError: expected ':' (2849319849.py, line 14)

In [136]:
#kernel = GaussianFilter1D(size=3, sigma=1.0)
gaussian2D = GaussianFilterND(
    size = numpy.array([3, 3]),
    mu = numpy.array([0.0, 0.0]),
    sigma = numpy.array([1.0, 1.0])
)

# nd = SeparableGaussianFilterND(
#     size = numpy.array([3, 3]),
#     sigmas = numpy.array([ 1.0, 1.0 ])
# )

print(gaussian2D.kernel)

#print(kernel.array)
#print(kernel(numpy.ones(4)))
#print(grid)

# print(numpy.reshape(numpy.arange(0, 16), (4, 4))[::-1, ::-1])

TypeError: Can't instantiate abstract class GaussianFilterND with abstract method __call__

In [127]:
import lasp.filters.linear
lasp.filters.linear.gaussian_filter(size=3, sigma=1.0)

array([[0.05854983, 0.09653235, 0.05854983],
       [0.09653235, 0.15915494, 0.09653235],
       [0.05854983, 0.09653235, 0.05854983]])

In [128]:
ones = numpy.ones((5, 5))
ones

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [129]:
convolve2D_full(ones, gaussian2D.kernel)

array([[0.05854983, 0.15508218, 0.21363202, 0.21363202, 0.21363202,
        0.15508218, 0.05854983],
       [0.15508218, 0.41076948, 0.56585166, 0.56585166, 0.56585166,
        0.41076948, 0.15508218],
       [0.21363202, 0.56585166, 0.77948368, 0.77948368, 0.77948368,
        0.56585166, 0.21363202],
       [0.21363202, 0.56585166, 0.77948368, 0.77948368, 0.77948368,
        0.56585166, 0.21363202],
       [0.21363202, 0.56585166, 0.77948368, 0.77948368, 0.77948368,
        0.56585166, 0.21363202],
       [0.15508218, 0.41076948, 0.56585166, 0.56585166, 0.56585166,
        0.41076948, 0.15508218],
       [0.05854983, 0.15508218, 0.21363202, 0.21363202, 0.21363202,
        0.15508218, 0.05854983]])

In [137]:
import scipy.signal
scipy.signal.convolve2d(ones, gaussian2D.kernel, mode='valid')

array([[0.77948368, 0.77948368, 0.77948368],
       [0.77948368, 0.77948368, 0.77948368],
       [0.77948368, 0.77948368, 0.77948368]])